# Clopath plasticity (voltage-based STDP)

Tutor: Johanna Senk [<span>&#9993;</span>](mailto:j.senk@sussex.ac.uk)  
PLENA 2024

We use the synapse model `clopath_synapse`. Compatible neuron models are `aeif_psc_delta_clopath` (used here) and `hh_psc_alpha_clopath`; these neuron models are capable of archiving continuous quantities.  
For details, refer to the NEST documentation: [clopath_synapse](https://nest-simulator.readthedocs.io/en/stable/models/clopath_synapse.html), [aeif_psc_delta_clopath](https://nest-simulator.readthedocs.io/en/stable/models/aeif_psc_delta_clopath.html), and [hh_psc_delta_clopath](https://nest-simulator.readthedocs.io/en/stable/models/hh_psc_delta_clopath.html).

This notebook is based on the NEST example
[Clopath Rule: Bidirectional connections](https://nest-simulator.readthedocs.io/en/stable/auto_examples/clopath_synapse_small_network.html).  
An earlier version of the notebook was developed by Agnes Korcsak-Gorzo for BNNI 2022.

## Background

Clopath plasticity accounts for non-linear effects of spike frequency on weight changes which had been previously observed in experiments (Sjöström et al., 2001). It does so by using the evolution of the postsynaptic membrane voltage around postsynaptic spike events instead of the postsynaptic spikes themselves. This requires a neuron model that takes into account features of membrane potential excursions near spike events, such as modified adaptive exponential integrate-and-fire (aeif) model neurons that are used in the original publication (Clopath et al., 2010, see section 5.2) or Hodgkin-Huxley neurons.

<img src="MorrisonEtAl2008_Fig2B.png" alt="Morrison et al. (2008) Fig. 2B" width="50" align="center"/> 

*Morrison et al. (2008) Fig. 2B* 

We assume the following equation to be the general form for plasticity models. The change of a weight of a synapse from neuron $j$ to neuron $i$ is given by:
\begin{align}
\frac{dW_{ij}(t)}{dt} = F(W_{ij}(t), s^*_i (t), s^*_j (t), V^*_i (t))
\end{align}
The superscript $*$ indicates causal functionals, potentially depending on all past values. $s(t)$ are spike trains and $V_i$ is the postsynaptic membrane potential.

Here, the right-hand side is the sum of $F_\text{LTD}$ and $F_\text{LTP}$ as defined below (with the pre-factors $A_\text{LTD}$ and $A_\text{LTP}$ controlling the relative strengths of these contributions):

Long-term depression (LTD):
\begin{align}
F_\text{LTD} ( s_j (t),\,V^*_{i,\text{LTD}} (t)) &= -A_\text{LTD} s_j (t) V^*_{i,\text{LTD}}\\
\text{with } V^*_{i,\text{LTD}} &= (\overline{u}_- -\theta_-)_+,\\
\overline{u}_-(t) &= (\kappa_- * V_i) (t - d_s)
\end{align}

Long-term potentiation (LTP):
\begin{align}
F_\text{LTP} ( s_j^* (t),\,V^*_{i,\text{LTP}} (t)) &= A_\text{LTP} s_j^* (t) V^*_{i,\text{LTP}}\\
\text{with } s^*_j &= \kappa_s * s_j\\
V^*_{i,\text{LTP}} &= (\overline{u}_+ - \theta_-)_+ (V_i - \theta_+)_+\\
\overline{u}_+ (t) &= (\kappa_+ * V_i) (t - d_s)
\end{align}

The above equations use the threshold linear function (with $H$ being the Heaviside step function):
\begin{align}
(x-x_0)_+ = H (x-x_0) (x-x_0)
\end{align}

$\kappa_\pm$ are exponential kernels applied to the postsynaptic membrane potential and $\kappa_s$ is applied to the presynaptic spike train:
\begin{align}
\kappa (t) = H(t) \frac{1}{\tau} \exp(-\frac{1}{\tau})
\end{align}

The time-independent parameters θ± serve as thresholds below which the (low-pass filtered) membrane potential does not cause any weight change.

### References

**Plasticity mechanism**

*Clopath C, Büsing L, Vasilaki E, Gerstner W (2010). Connectivity reflects coding: a model of voltage-based STDP with homeostasis. Nature Neuroscience 13:3, 344--352. [doi:10.1038/nn.2479](https:doi.org/10.1038/nn.2479)*

**NEST implementation**

*J. Stapmanns, J. Hahne, M. Helias, M. Bolten, M. Diesmann, D. Dahmen (2021). Event-Based Update of Synapses in Voltage-Based Learning Rules. Frontiers in Neuroinformatics, 15:609147. [doi: 10.3389/fninf.2021.609147](https://doi.org/10.3389/fninf.2021.609147)*

## This notebook

In this experiment, we simulate a small network of ten excitatory and three
inhibitory ``aeif_psc_delta_clopath`` neurons. The neurons are randomly connected
and driven by 500 Poisson generators. The synapses from the Poisson generators
to the excitatory population and those among the neurons of the network
are plastic Clopath synapses (indicated in blue). The rate of the Poisson generators is modulated with
a Gaussian profile whose center shifts randomly each 100 ms between ten
equally spaced positions.
This setup is inspired by Fig. 5 in Clopath et al. (2010), see also Fig. 7 of Stapmanns et al. (2021). We will demonstrate that the Clopath synapse is able to establish
bidirectional connections.

<img src="stapmanns2021event_fig7a_setup-clopath.png" alt="sketch" width="300" align="center"/>

*Stapmanns et al. (2021) Fig. 7C*

## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({
    'font.size': 16,
    })
import numpy as np
import random
import nest
nest.set_verbosity('M_ERROR')

Set simulation parameters:

In [ ]:
resolution = 0.1             # simulation step size (ms)
simulation_time = 1e4       # simulation time
sim_interval = 100.         # simulate in intervals of 100 ms for shifting the Gaussian

Check the default values of the neuron model and set the values used here:

In [ ]:
nrn_model = 'aeif_psc_delta_clopath'
nest.GetDefaults(nrn_model)

In [ ]:
nrn_params = {
    # dynamic state variables
    'V_m': -30.6,             # initial membrane potential (mV)
    'w': 0.,                  # spike-adaptation current (pA)
    
    # membrane parameters
    'g_L': 30.,               # leak conductance (nS)
    'tau_u_bar_plus': 7.,     # time constant of the low-pass filtered membrane potential (ms)
    'tau_u_bar_minus': 10.,   # time constant of the low-pass filtered membrane potential (ms)
    'C_m': 281.,              # membrane capacitance (pF)
    
    # spike-adaptation parameters
    'tau_w': 144.,            # adaptation time constant (ms)
    'a': 4.,                  # sub-threshold adaptation (nS)
    'Delta_T': 2.,            # slope factor (mV)
    'V_peak': 20.,            # spike detection threshold (mV)
    'b': .0805,               # spike-triggered adaptation (pA)

     # Clopath rule parameters
    'u_ref_squared': 60.**2,  # reference value for u_bar_bar^2
    'A_LTP': 8e-6,            # amplitude of facilitation (1/mV^2)
    'A_LTD': 14e-6,           # amplitude of depression (1/mV)
    'A_LTD_const': False,     # flag that indicates whether A_LTP should be constant (True) or multiplied
                              # by u_bar^2 / u_ref_squared (False)
    
    # other parameters
    't_clamp': 2.,            # duration of clamping of membrane potential after a spike (ms)
    }

Set network parameters:

In [ ]:
N_exc = 10   # number exc. neurons
N_inh = 3    # number inh. neurons
N_pg = 500   # number Poisson generators

Set connection parameters:

In [ ]:
w_exc_exc = 0.25
delay = resolution

Set external input parameters:

In [ ]:
pg_A = 30.      # amplitude of Gaussian
pg_sigma = 10.  # std deviation of Gaussian

Clear NEST memory:

In [ ]:
nest.ResetKernel()
nest.resolution = resolution

Create neurons and devices:

In [ ]:
pop_exc = nest.Create(nrn_model, N_exc, nrn_params)
pop_inh = nest.Create(nrn_model, N_inh, nrn_params)

pg = nest.Create('poisson_generator', N_pg)

# create intermediate parrot neurons since Poisson generators can only be connected
# with static connections
pop_input = nest.Create('parrot_neuron', N_pg)

Connect the network:

In [ ]:
syn_spec_static = {'synapse_model': 'static_synapse', 'weight': 1., 'delay': delay}
conn_spec_ata = {'rule': 'all_to_all', 'allow_autapses': False}

# set the maximum allowed weight
nest.CopyModel('clopath_synapse', 'plastic_input_exc', {'Wmax': 3.})
nest.CopyModel('clopath_synapse', 'plastic_exc_exc', {'Wmax': 0.75})

nest.Connect(pg, pop_input, 'one_to_one', syn_spec_static)

nest.Connect(pop_input, pop_exc, conn_spec_ata,
             {'synapse_model': 'plastic_input_exc',
              'weight': nest.random.uniform(0.5, 2.), 'delay': delay})

nest.Connect(pop_input, pop_inh, conn_spec_ata,
             {'synapse_model': 'static_synapse',
              'weight': nest.random.uniform(0., 0.5), 'delay': delay})

nest.Connect(pop_exc, pop_exc, conn_spec_ata,
             {'synapse_model': 'plastic_exc_exc',
              'weight': w_exc_exc, 'delay': delay})

nest.Connect(pop_exc, pop_inh, {'rule': 'fixed_indegree', 'indegree': 8},
             syn_spec_static)

nest.Connect(pop_inh, pop_exc, {'rule': 'fixed_outdegree', 'outdegree': 6},
             syn_spec_static)

Randomize the initial membrane potential:

In [ ]:
pop_exc.V_m = nest.random.normal(-60., 25.)
pop_inh.V_m = nest.random.normal(-60., 25.) 

Simulate in intervals, changing the position of the Poisson generator:

In [ ]:
for i in range(int(simulation_time / sim_interval)):
    rates = np.empty(500)  # set rates of poisson generators
    pg_mu = 25 + random.randint(0, 9) * 50  # pg_mu will be randomly chosen out of 25,75,125,...,425,475
    for j in range(500):
        rates[j] = pg_A * np.exp((-1 * (j - pg_mu)**2) / (2 * pg_sigma**2))
        pg[j].rate = rates[j] * 1.75
    nest.Simulate(sim_interval)

Get results for weights of the synapses within the exc. population:

In [ ]:
# sort weights according to sender and reshape
exc_conns = nest.GetConnections(pop_exc, pop_exc)
idx_array = np.argsort(np.array(exc_conns.source))
targets = np.reshape(np.array(exc_conns.target)[idx_array], (N_exc, N_exc - 1))
weights = np.reshape(np.array(exc_conns.weight)[idx_array], (N_exc, N_exc - 1))

# sort weights according to target
for i, (trgs, ws) in enumerate(zip(targets, weights)):
    idx_array = np.argsort(trgs)
    weights[i] = ws[idx_array]

weight_matrix = np.zeros((N_exc, N_exc))

tu10 = np.triu_indices_from(weight_matrix, 1)
tu9 = np.triu_indices_from(weights)
weight_matrix[tu10[0], tu10[1]] = weights[tu9[0], tu9[1]]

tl10 = np.tril_indices_from(weight_matrix, -1)
tl9 = np.tril_indices_from(weights, -1)
weight_matrix[tl10[0], tl10[1]] = weights[tl9[0], tl9[1]]

# calculate difference between initial and final value
init_w_matrix = (np.ones((N_exc, N_exc)) - np.identity(N_exc)) * w_exc_exc
diff_weight_matrix = weight_matrix - init_w_matrix

Plot results:

In [ ]:
fig, axes = plt.subplots(1, 3, sharex=False, figsize=(15,4))

def plot_weights(ax, weight_matrix, label, ylabel=False, vmin=None):
    cax = ax.imshow(weight_matrix, vmin=vmin)
    cbarB = fig.colorbar(cax, ax=ax)
    cbarB.set_label(label)
    ax.set_xlabel('target neuron')
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    ax.set_xticks(range(0, 10, 2))
    ax.set_yticks(range(0, 10, 2))
    if ylabel:
        _ = ax.set_ylabel('source neuron')

axes[0].set_title('initial weights')
plot_weights(axes[0], init_w_matrix, r'$w$', ylabel=True, vmin=0.2)
axes[1].set_title('final weights')
plot_weights(axes[1], weight_matrix, r'$w$', vmin=0.2)
axes[2].set_title('weight difference')
plot_weights(axes[2], diff_weight_matrix, r'$\Delta w$')

fig.tight_layout()

You may also want to check out another NEST example with Clopath plasticity: [Clopath Rule: Spike pairing experiment](https://nest-simulator.readthedocs.io/en/stable/auto_examples/clopath_synapse_spike_pairing.html#auto-examples-clopath-synapse-spike-pairing)